In [1]:
import numpy as np

np.zeros((5, 6)) # матрица 5 строк 6 столбцов из нулей

X = np.random.normal(size=(3,3))
X @ X # с помощью собаки можно перемножать матрицы

np.linalg.det(X)

-1.03619806227534

In [4]:
import pandas as pd
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_csv('train.zip', compression = 'zip', header = 0, sep = ',' , quotechar = '"')
df = df.drop(labels = 'dropoff_datetime', axis = 1)
# при наличии в pickup_datetime данных из прошлого нельзя получить train и тест выборки
# просто с помощью train_test_split тк мы хотим обучить модель на ПРОШЛЫХ данных, чтобы она
# предсказывала БУДУЩЕЕ
df = df.sort_values(by = 'pickup_datetime')
df
train_df, test_df = df[:10 ** 6], df[10 ** 6:]

train_df['log_trip_duration'] = np.log1p(train_df.trip_duration)
test_df['log_trip_duration'] = np.log1p(test_df.trip_duration)

train_df.pickup_datetime = pd.to_datetime(train_df.pickup_datetime)
date = train_df.pickup_datetime.apply(lambda x: x.date())
# plt.figure(figsize = (25,5))
# date_count_plot = sns.countplot(x = date)
# date_count_plot.set_xticklabels(date_count_plot.get_xticklabels(), rotation = 90)


# разделение на 2 матрицы (признаков и весов) и обучение алгоритма линейной регрессии 
# с помощью функции

def create_features(data_frame):
    X = pd.concat([
        data_frame.pickup_datetime.apply(lambda x: x.timetuple().tm_yday),
        data_frame.pickup_datetime.apply(lambda x: x.hour)
    ], axis = 1, keys = ['day', 'hour'])
    
    return X, data_frame.log_trip_duration

X_train, y_train = create_features(train_df)
X_test, y_test = create_features(test_df)

ohe = OneHotEncoder(sparse = False, categorical_features = [1])
X_train = ohe.fit_transform(X_train) # обучение для преобразования данных
X_test = ohe.transform(X_test)

# теперь после преобразования даты формата гг-мм-дд в день в году (цифра), мы готовы к обучению
# алгоритма линейной регрессии: 

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
predict = lin_reg.predict(X_test)
mse = mean_squared_error(y_true = y_train, y_pred = predict)
mse

C:\Users\Gleb\AppData\Local\Temp/ipykernel_16412/2688409035.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['log_trip_duration'] = np.log1p(train_df.trip_duration)
C:\Users\Gleb\AppData\Local\Temp/ipykernel_16412/2688409035.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['log_trip_duration'] = np.log1p(test_df.trip_duration)
c:\users\gleb\pycharmprojects\pythonproject1\venv\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a cop

AttributeError: 'Timestamp' object has no attribute 'Timestamp'

In [39]:
6/3

2.0

In [8]:
%%latex


UsageError: %%latex is a cell magic, but the cell body is empty.


In [19]:
%%latex # ФОРМУЛА(1)
$$w^{t+1} = w^t - a_t\nabla Q(w^{t})$$ # правило обновления весов в градиентном спуске

<IPython.core.display.Latex object>

In [20]:
%%latex # ФОРМУЛА(2) # проверка для остановки в градиенте
$$w^{t} - w^{t-1} < e$$

<IPython.core.display.Latex object>

In [38]:
%%latex # ФОРМУЛА(3) # вычисление компоненты градиента (ещё раз разобрать вывод, непонятно)
$$\frac{\partial Q(w)}{w_j} = \frac{2}{l}\sum_{i=1}^lx_{ij}(\langle w,x_i\rangle - y_i)$$

<IPython.core.display.Latex object>

In [ ]:
# Градиентный спуск для линейной регресси:
from sklearn.base import BaseEstimator

class LinearRegression(BaseEstimator): 
    # инициализация
    # epsilon - разница для нормы изменения весов (условие для остановки)
    # w0 - начальные веса (np.array(...))
    # max_steps - макс кол-во шагов в градиентном спуске
    # aplha - шаг градиентного спуска (шаг обучения)
    def __init__(self, epsilon=1e-4, w0 = None, max_steps = 1000, alpha=1e-2):
        self.epsilon = epsilon
        self.w0 = w0
        self.max_steps = max_steps
        self.alpha = alpha
        self.w = None # итоговые веса
        self.w_history = []
    # обучение линейной модели
    def fit(self, X, y):
        # X - np.array(l,d)
        # y - np.array(l)
        l, d = X.shape
        # задаём начальные веса:
        if self.w0 == None:
            self.w0 = np.zeros(d)
        self.w = w0
        # далее итерируем градиентный спуск по кол-ву шагов max_step
        for step in range(self.max_steps):
            self.w_history.append(self.w)
            # по ФОРМУЛЕ(1) вычисляем новое значение весов
            w_new = self.w - self.alpha * self.calc_gradient(X, y)
            # проверяем условие на разность весов по ФОРМУЛЕ(2)
            # считаем норму, тк имеем дело с векторами (w - вектор весов)
            if np.linalg.norm(self.w_new - self.w) < epsilon:
                break
        self.w = self.w_new
        return self
    
    def calc_gradient(self, X, y):
        l, d = X.shape
        gradient = []
        
        for j in range(d):
            dQ = 0
            for i in range(l):
                dQ += (2/l) * X[i][j] * (np.dot(X[i], self.w) - y[i])
            gradient.append(dQ)
            
        return np.array(gradient)
    
    def predict(self, X):
        if self.w == None:
            raise Exception 'Модель не обучена'
        l, d = X.shape
        y_pred = []
        for i in range(l):
            y_pred.append(np.dot(X[i], self.w))
        return np.array(y_pred)
            
            

In [21]:
%%latex #формула градиентного спуска в матричной форме:
$$\nabla Q(w) = \frac{2}{l}X^{T}(X_w - y)$$

<IPython.core.display.Latex object>

In [ ]:
# линейная регрессия в матричном (векторном виде)
# матричная форма намного быстрее обычного градиента
# всё, что меняется - это формула градиента в матричной форме
class LinearRegressionVectorized(BaseEstimator): 

    def __init__(self, epsilon=1e-4, w0 = None, max_steps = 1000, alpha=1e-2):
        self.epsilon = epsilon
        self.w0 = w0
        self.max_steps = max_steps
        self.alpha = alpha
        self.w = None
        self.w_history = []
    def fit(self, X, y):
        l, d = X.shape
        if self.w0 == None:
            self.w0 = np.zeros(d)
        self.w = w0
        for step in range(self.max_steps):
            self.w_history.append(self.w)
            w_new = self.w - self.alpha * self.calc_gradient(X, y)
            if np.linalg.norm(self.w_new - self.w) < epsilon:
                break
        self.w = self.w_new
        return self
    
    def calc_gradient_vectorized(self, X, y):
        l, d = X.shape
        return (2/l)*np.dot(X.T, (np.dot(X, self.w) - y))
        
    def predict(self, X):
        if self.w == None:
            raise Exception 'Модель не обучена'
        return np.dot(X, self.w)